# Divide-and-Conquer Paper

In [1]:
# install using `conda install -c conda-forge line_profiler`
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.utils.estimator_checks import check_estimator
from sklearn.utils.validation import check_is_fitted
import numpy as np
from numpy.linalg import LinAlgError
# set global seed
# np.random.seed(123)
from sklearn.model_selection import GridSearchCV, RepeatedKFold
from sklearn.metrics import r2_score, accuracy_score

import pandas as pd
from datetime import datetime
from copy import deepcopy
# utils for plotting
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.colors as colors
from joblib import Parallel, delayed
import pickle

from functools import reduce
from operator import concat

# utils for timing
from goodpoints.tictoc import tic, toc, TicToc
# utils for kernel ridge regression
from goodpoints.krr.util_estimators import get_estimator, get_sigma_heuristic
# utils for evaluating kernels
from goodpoints.krr.thin.util_k_mmd import kernel_eval, to_regression_kernel
# utils for generate samples from the data distribution
from goodpoints.krr.util_sample import get_Xy, ToyData , get_toy_dataset #, get_housing_dataset
# utils for dataset thinning
from goodpoints.krr.thin.util_thin import sd_thin, kt_thin2
# # falkon baseline
# from goodpoints.krr.falkon.util_falkon_estimators import KernelRidgeFalkon

`eigenpro2` is not installed...
Using `torch.linalg.solve` for training the kernel model

          and may cause an `Out-of-Memory` error
`eigenpro2` is a more scalable solver. To use, pass `method="eigenpro"` to `model.fit()`
To install `eigenpro2` visit https://github.com/EigenPro/EigenPro-pytorch/tree/pytorch/


In [3]:
# add this to be able to render plotly plots in non-vscode notebooks
import plotly.io as pio
pio.renderers.default = "notebook_connected"

## Setup

In [4]:
### Toy dataset parameters

n = 2**13

X_name = 'unif-[0,1]' # 'unif-[0,1]'  # ['unif', 'unif-[0,1]', 'gauss', 'mog']
f_name = 'dnc-paper' # 'dnc-paper' # ['sin', 'stair', 'quad', 'sum-gauss', 'sum-laplace', 'step', 'dnc-paper']
noise = 0.1 # np.sqrt( 0.2 ) # in paper, variance is 1/5

d = 1
# M = 2
k = 8 # number of anchor points for sum-gauss and sum-laplace

### Regression parameters

kernel = 'sobolev'  # ['sobolev', 'gauss', 'laplace']
sigma =  0.25
alpha_pow = -2/3  # gauss = -1, sobolev = -2/3
alpha = np.power(float(n),alpha_pow)

### Experiment parameters

# varying_variable = 'kernel' # ['kernel', 'd', 'M']
n_repeats = 20
save = False
logn_lo = 8
logn_hi = 14
use_cross_validation = False
n_jobs = 1
# method = 'st'

### Thinning parameters

# m = None # Thinned dataset will have size n/2**m
m=10

In [5]:
# Parameters
n_jobs = 8
n_repeats = 100
save = True
kernel = "gauss"
X_name = "unif"
f_name = "sum-gauss"
alpha_pow = -1


In [6]:
print('alpha', alpha)

alpha 0.0024607833005759246


In [7]:
# Determine auxiliary parameters
task = 'regression'
refit = 'neg_mean_squared_error'
postprocess = None

filename = '_'.join([X_name, f_name, f'k={k}', f'noise={noise}'])
print(filename)

baseline_loss = noise**2
print(f'baseline loss: {baseline_loss}')

unif_sum-gauss_k=8_noise=0.1
baseline loss: 0.010000000000000002


In [8]:
toy_data_noise = ToyData(X_name, f_name, 
                        #  X_var=X_var, 
                         noise=noise, 
                        d=d, 
                        # M=M, 
                        k=k)

X_train, y_train = toy_data_noise.sample(n)
X_test, y_test = toy_data_noise.sample(10000)
# validation set used for cross validation
X_val, y_val = toy_data_noise.sample(10000)

# no noise version for plotting purposes
toy_data_no_noise = ToyData(X_name, f_name, noise=0, 
                    d=d, k=k)
X_no_noise, y_no_noise = toy_data_no_noise.sample(n)

print(X_train.shape, y_train.shape)

(8192, 1) (8192,)


In [9]:
np.std(X_train), np.std(y_train)

(1.00012207776399, 0.8509599622700201)

In [10]:
def trace_Xy(X, y, name=None, color=None, alpha=0.5):
    d = X.shape[-1]

    if d==1:
        return go.Scatter(
            x=X.squeeze(), 
            y=y, 
            mode='markers',
            name=name,
            opacity=alpha,
            marker=dict(
                color=color,
            )
        )

    elif d==2:
        x1,x2 = X[:,0], X[:,1]
        return go.Scatter3d(x=x1, y=x2, z=y, mode='markers', name=name, marker=dict(
            symbol='circle',
            opacity=alpha,
            color=color,
            size=2,
            # line=dict(width=1),
        ))

    else:
        print(f"cannot plot data with dimension {d}")

In [11]:
# def hnorm(k):
#     from goodpoints.krr.thin.util_k_mmd import laplacian, gauss
#     anchor_points = np.linspace(-1, 1, k)[:, np.newaxis]
#     # print(anchor_points)

#     if f_name == 'sum-gauss':
#         K = gauss(anchor_points, anchor_points, 0.25)
#     elif f_name == 'sum-laplace':
#         K = laplacian(anchor_points, anchor_points, 0.25)
#     else:
#         raise ValueError(f'f_name {f_name} not supported')
#     # print(K)
    
#     return np.sum(K)
#     # return 1/(k**2) * np.sum(K)

In [12]:
# hnorm(2)

In [13]:
fig = go.Figure(data=[
    trace_Xy(X_train, y_train, name='train'),
    # trace_Xy(X_test, y_test, name='test'),
    trace_Xy(X_no_noise, y_no_noise, name='no noise', color='black'),
])
fig.update_layout(
    title=f'X={X_name}, f={f_name}, std[f]={np.std(y_train):.4f}', #, hnorm[f]={hnorm(k):.4f}',
    height=400,
    width=800,
)
fig.show()
# save fig
if save: fig.write_image(f"figures/{filename}_function.png")

In [14]:
np.minimum(np.array([0,1])[:,np.newaxis],np.array([1,2,3])[:,np.newaxis].T)

array([[0, 0, 0],
       [1, 1, 1]])

## KRR Full

In [15]:
krr_full = get_estimator(
    task,
    'full',
    kernel=kernel, 
    sigma=sigma, 
    alpha=alpha
)

In [16]:
krr_full.fit(X_train, y_train)

array([[1.00000000e+00, 9.10743409e-08, 2.95922371e-04, ...,
        7.77013228e-27, 6.93645568e-05, 9.14453373e-01],
       [9.10743409e-08, 1.00000000e+00, 2.50919016e-01, ...,
        9.25548012e-07, 4.19662279e-01, 7.49404426e-09],
       [2.95922371e-04, 2.50919016e-01, 1.00000000e+00, ...,
        3.62334583e-11, 9.42191530e-01, 4.91952826e-05],
       ...,
       [7.77013228e-27, 9.25548012e-07, 3.62334583e-11, ...,
        1.00000000e+00, 3.73678717e-10, 6.87993203e-29],
       [6.93645568e-05, 4.19662279e-01, 9.42191530e-01, ...,
        3.73678717e-10, 1.00000000e+00, 9.96549677e-06],
       [9.14453373e-01, 7.49404426e-09, 4.91952826e-05, ...,
        6.87993203e-29, 9.96549677e-06, 1.00000000e+00]])

In [17]:
pred_full = krr_full.predict(X_test)
train_pred_full = krr_full.predict(X_train)

print('train MSE:', mean_squared_error(y_train, train_pred_full))
print('MSE:', mean_squared_error(y_test, pred_full))

train MSE: 0.009783793706406373
MSE: 0.009834480146949418


### sanity check

In [18]:
# make sure krr full is same as krr dnc (st) with 1 partition
krr_sd_thin_0 = get_estimator(
    task,
    'st',
    kernel=kernel, 
    sigma=sigma, 
    alpha=alpha,
    m=0,
    use_dnc=True
)

In [19]:
krr_sd_thin_0.fit(X_train, y_train)

In [20]:
print('train MSE:', mean_squared_error(y_train, krr_sd_thin_0.predict(X_train)))
print('MSE:', mean_squared_error(y_test, krr_sd_thin_0.predict(X_test)))

train MSE: 0.009783793706406373


MSE: 0.009834480146949418


In [21]:
krr_kt_thin_0 = get_estimator(
    task,
    'kt',
    kernel=kernel, 
    sigma=sigma, 
    alpha=alpha,
    m=0,
    use_dnc=True,
    use_compresspp=False,
)

In [22]:
krr_kt_thin_0.fit(X_train, y_train)

In [23]:
print('train MSE:', mean_squared_error(y_train, krr_kt_thin_0.predict(X_train)))
print('MSE:', mean_squared_error(y_test, krr_kt_thin_0.predict(X_test)))

train MSE: 0.009783793706406373


MSE: 0.009834480146949418


In [24]:
fig = go.Figure(data=[
    # trace_Xy(krr_sd_thin.X_fit_, krr_sd_thin.y_fit_, name='train coreset', alpha=1),
    trace_Xy(X_test, y_test, name='test', alpha=0.1),
    trace_Xy(X_test, pred_full, name='pred', alpha=0.4)
])
fig.update_layout(
    title='full',
    height=400,
    width=800,
)
fig.show()
# save fig
if save: fig.write_image(f"figures/{filename}_full.png")

## DnC (Standard Thin)

In [25]:
print(m)

10


In [26]:
krr_sd_thin = get_estimator(
    task, 
    'st', 
    alpha=alpha, # / np.power(n, 1/4), 
    kernel=kernel, 
    sigma=sigma, 
    m=m, 
    use_dnc=True,
)

In [27]:
%%time
krr_sd_thin.fit(X_train, y_train)

coreset size: 8, m: 10


CPU times: user 130 ms, sys: 0 ns, total: 130 ms
Wall time: 130 ms


In [28]:
# krr_sd_thin.estimators_[0].sol_
# krr_sd_thin.estimators_[0].y_fit_
# krr_sd_thin.estimators_[0].X_fit_
# a = (k+alpha)^-1 y
# 0.63134153/(1+0.09131417 + alpha)

In [29]:
%%time
pred_sd = krr_sd_thin.predict(X_test)
train_pred_sd = krr_sd_thin.predict(X_train)

print('train MSE:', mean_squared_error(y_train, train_pred_sd))
print('MSE:', mean_squared_error(y_test, pred_sd))

train MSE: 0.1648284551612476
MSE: 0.1639038939124429
CPU times: user 6.89 s, sys: 165 ms, total: 7.06 s
Wall time: 883 ms


In [30]:
fig = go.Figure(data=[
    # trace_Xy(krr_sd_thin.X_fit_, krr_sd_thin.y_fit_, name='train coreset', alpha=1),
    trace_Xy(X_test, y_test, name='test', alpha=0.1),
    trace_Xy(X_test, pred_sd, name='pred', alpha=0.4)
])
fig.update_layout(
    title='st',
    height=400,
    width=800,
)
fig.show()
# save fig
if save: fig.write_image(f"figures/{filename}_st.png")

## DnC (Kernel Thin)

In [31]:
n

8192

In [32]:
krr_kt_thin = get_estimator(
    task, 
    'kt', 
    alpha=alpha, # / np.power(n, 1/4), 
    kernel=kernel, 
    sigma=sigma, 
    m=m, 
    use_dnc=True,
    use_compresspp=False,
)

In [33]:
%%time
krr_kt_thin.fit(X_train, y_train)

CPU times: user 1.1 s, sys: 164 ms, total: 1.27 s
Wall time: 1.27 s


In [34]:
%%time
pred_kt = krr_kt_thin.predict(X_test)
train_pred_kt = krr_kt_thin.predict(X_train)

print('train MSE:', mean_squared_error(y_train, train_pred_kt))
print('MSE:', mean_squared_error(y_test, pred_kt))

train MSE: 0.12185177280103926
MSE: 0.12103367365779417
CPU times: user 6.94 s, sys: 181 ms, total: 7.12 s
Wall time: 890 ms


In [35]:
fig = go.Figure(data=[
    # trace_Xy(krr_sd_thin.X_fit_, krr_sd_thin.y_fit_, name='train coreset', alpha=1),
    trace_Xy(X_test, y_test, name='test', alpha=0.1),
    trace_Xy(X_test, pred_kt, name='pred', alpha=0.4)
])
fig.update_layout(
    title='kt',
    height=400,
    width=800,
)
fig.show()
# save fig
if save: fig.write_image(f"figures/{filename}_kt.png")

## Experiment

In [36]:
"""
Varying variables (during grid search, these are NOT parallelized)
"""

varying_variable = 'alpha'
varying_variable_values = ['under', 'not-under'] #[2.0**-logn for logn in np.arange(logn_lo, logn_hi, 2)]

In [37]:
print('Running experiment with varying variable:', varying_variable)
print('taking values:', varying_variable_values)

Running experiment with varying variable: alpha
taking values: ['under', 'not-under']


In [38]:
# # Default param grid to search for each model
default_param_grid = {
    "sigma" :   [sigma,],
    # "alpha" :   [1e1, 1e-0, 1e-1, 1e-2, 1e-3, 1e-4],

    # "sigma" :   1/np.sqrt(2*np.array([0.5, 1., 2, 5.])),
    # "alpha" :   [0.01, 0.02]
}
falkon_param_grid = {
    "sigma" :   [sigma,],
    # "alpha" :   [1e-4, 1e-5,1e-6, 1e-7, 0],
}


# Model constructors and data size for each model
# We allow for different data sizes to avoid running Full KR on large datasets
model_configs = {
    # 'full' : {
    #     'logn' : np.arange(logn_lo, logn_hi, 2),
    #     # 'logn' : np.arange(8, 13, 2),
    #     'kwargs': {
    #         'postprocess' : postprocess,
    #         # 'alpha' : alpha,
    #         # 'sigma' : sigma,
    #     },
    #     'param_grid' : default_param_grid,
    # },
    
}
for m in range(0, logn_hi, 2):
    # if method == 'st':
    model_configs[f'st_{m}'] = {
        'logn' : np.arange(logn_lo, logn_hi, 1),
        # 'logn' : np.arange(8, 13, 2),
        'kwargs' : {
            'm' : m,
            'postprocess' : postprocess,
            # 'alpha' : alpha,
            # 'sigma' : sigma,
            'use_dnc' : True,
        },
        'param_grid' : default_param_grid,
    }
    # elif method == 'kt':
    model_configs[f'kt_{m}'] = {
        'logn' : np.arange(logn_lo, logn_hi, 1),
        # 'logn' : np.arange(8, 13, 2),
        'kwargs' : {
            'm' : m,
            'postprocess' : postprocess,
            # 'alpha' : alpha,
            # 'sigma' : sigma,
            'use_dnc' : True,
            'use_compresspp' : False,
        },
        'param_grid' : default_param_grid,
    }
    # else:
    #     raise ValueError(f'unknown method {method}')

In [39]:
model_configs

{'st_0': {'logn': array([ 8,  9, 10, 11, 12, 13]),
  'kwargs': {'m': 0, 'postprocess': None, 'use_dnc': True},
  'param_grid': {'sigma': [0.25]}},
 'kt_0': {'logn': array([ 8,  9, 10, 11, 12, 13]),
  'kwargs': {'m': 0,
   'postprocess': None,
   'use_dnc': True,
   'use_compresspp': False},
  'param_grid': {'sigma': [0.25]}},
 'st_2': {'logn': array([ 8,  9, 10, 11, 12, 13]),
  'kwargs': {'m': 2, 'postprocess': None, 'use_dnc': True},
  'param_grid': {'sigma': [0.25]}},
 'kt_2': {'logn': array([ 8,  9, 10, 11, 12, 13]),
  'kwargs': {'m': 2,
   'postprocess': None,
   'use_dnc': True,
   'use_compresspp': False},
  'param_grid': {'sigma': [0.25]}},
 'st_4': {'logn': array([ 8,  9, 10, 11, 12, 13]),
  'kwargs': {'m': 4, 'postprocess': None, 'use_dnc': True},
  'param_grid': {'sigma': [0.25]}},
 'kt_4': {'logn': array([ 8,  9, 10, 11, 12, 13]),
  'kwargs': {'m': 4,
   'postprocess': None,
   'use_dnc': True,
   'use_compresspp': False},
  'param_grid': {'sigma': [0.25]}},
 'st_6': {'logn'

In [40]:
# Run experiment (depending on experiment_type)

results = []

i = 0
for name, config in model_configs.items():
    for logn in config['logn']:

        for v in varying_variable_values:
            kwargs = deepcopy(config['kwargs'])
            # kwargs[varying_variable] = v
            if v == 'under':
                kwargs['alpha'] = np.power(float(2**logn), alpha_pow)
            else:
                kwargs['alpha'] = np.power(float(2**logn / 2**kwargs['m']), alpha_pow)

            model_name = f"{name}_{v}"
            # trials = (1 if name in ['full'] else n_repeats)
            # st_0 is equivalent to KRR-Full
            trials = (1 if kwargs['m'] == 0 else n_repeats)

            X, y = get_toy_dataset(
                X_name=X_name,
                f_name=f_name,
                n=2**logn,
                # X_var=X_var,
                d=kwargs['d'] if 'd' in kwargs else d,
                noise=noise,
                # M=kwargs['M'] if 'M' in kwargs else M,
                k=k,
            )
            
            # Set kernel, alpha, sigma params
            if 'kernel' not in kwargs:
                kwargs['kernel'] = kernel
            
            model = get_estimator(task, name=name, **kwargs)
            if model is None: continue
            if kwargs['m'] > logn: continue

            print(f'i={i+1}: logn={logn}, model={model}')

            # STEP 2: Get optimal parameters through grid search
            # NOTE: we do something slightly better than k-fold cross validation.
            # Namely, we are trying to get rid of randomness in the Kernel Thinning (or Standard Thinning) routine,
            # but if we did 100-fold CV, then the validation set would be 1% of the data
            # (which is too small to get a good estimate of the validation score).
            # Instead we use the same train-val split for each parameter setting and repeat `trials` times
            if use_cross_validation:
                X_concat, y_concat = np.concatenate([X, X_val]), np.concatenate([y, y_val])
                split = [(np.arange(len(X)), np.arange(len(X), len(X)+len(X_val))) for _ in range(trials)]
                grid_search = GridSearchCV(
                    estimator=model,
                    param_grid=config['param_grid'],
                    return_train_score=True,
                    cv=split,
                    scoring=refit,
                    refit=False,
                    n_jobs=n_jobs,
                ).fit(X_concat, y_concat)
                # get validation scores
                cv_results = pd.DataFrame(grid_search.cv_results_)
                val_scores = []
                for i in range(trials):
                    val_scores.append( cv_results.iloc[grid_search.best_index_][f'split{i}_test_score'] )

                # get optimal parameters
                best_params = grid_search.best_params_
            else:
                # Dummy values
                val_scores = [1,] * trials
                
                best_params = {
                    'sigma' : sigma,
                    # 'alpha' : alpha, # * (len(X_train)**(1/4) if name in ['st', 'kt'] else 1),
                }    

            def run():
                # print(f"best params: {best_params}")            
                best_model = get_estimator(task, name=name, 
                                        sigma=best_params['sigma'],
                                        #    alpha=best_params['alpha'],
                                        **kwargs)
                best_model.fit(X, y)
                return best_model.predict(X_test).squeeze()

            parallel = Parallel(n_jobs=n_jobs)
            output_generator = parallel(delayed(run)() for _ in range(trials))

            mean_scores = []
            for test_pred in output_generator:
                if refit == 'neg_mean_squared_error':
                    test_scores = mean_squared_error(y_test, test_pred)
                elif refit == 'accuracy':
                    test_scores = accuracy_score(y_test, test_pred)
                else:
                    raise ValueError(f"invalid refit metric: {refit}")

                mean_scores.append( np.mean(test_scores) )

            results.append({
                "logn": logn, 
                "model": model_name, 
                "cv_results": pd.DataFrame(grid_search.cv_results_) if use_cross_validation else None,
                "best_index_" : grid_search.best_index_ if use_cross_validation else 0,
                "mean_scores" : mean_scores,
                # "std_score" : np.std(mean_scores),
            })

            if save:
                with open(f"results/{filename}.pkl", 'wb') as f:
                    pickle.dump(results, f)

            i += 1

sampling dataset with params ToyData(X_name=unif, f_name=sum-gauss, X_var=1, d=1, noise=0.1, M=4, k=8)
i=1: logn=8, model=KernelRidgeSTRegressor(alpha=0.00390625, kernel='gauss', m=0, use_dnc=True)


i=2: logn=8, model=KernelRidgeSTRegressor(alpha=0.00390625, kernel='gauss', m=0, use_dnc=True)


sampling dataset with params ToyData(X_name=unif, f_name=sum-gauss, X_var=1, d=1, noise=0.1, M=4, k=8)
i=3: logn=9, model=KernelRidgeSTRegressor(alpha=0.001953125, kernel='gauss', m=0, use_dnc=True)


i=4: logn=9, model=KernelRidgeSTRegressor(alpha=0.001953125, kernel='gauss', m=0, use_dnc=True)


sampling dataset with params ToyData(X_name=unif, f_name=sum-gauss, X_var=1, d=1, noise=0.1, M=4, k=8)
i=5: logn=10, model=KernelRidgeSTRegressor(alpha=0.0009765625, kernel='gauss', m=0, use_dnc=True)


...

i=140: logn=13, model=KernelRidgeSTRegressor(alpha=0.5, kernel='gauss', m=12, use_dnc=True)


i=141: logn=12, model=KernelRidgeKTRegressor(alpha=0.000244140625, kernel='gauss', m=12,
                       use_compresspp=False, use_dnc=True)


i=142: logn=12, model=KernelRidgeKTRegressor(alpha=1.0, kernel='gauss', m=12, use_compresspp=False,
                       use_dnc=True)


i=143: logn=13, model=KernelRidgeKTRegressor(alpha=0.0001220703125, kernel='gauss', m=12,
                       use_compresspp=False, use_dnc=True)


i=144: logn=13, model=KernelRidgeKTRegressor(alpha=0.5, kernel='gauss', m=12, use_compresspp=False,
                       use_dnc=True)


In [41]:
results

[{'logn': 8,
  'model': 'st_0_under',
  'cv_results': None,
  'best_index_': 0,
  'mean_scores': [0.010716326271436491]},
 {'logn': 8,
  'model': 'st_0_not-under',
  'cv_results': None,
  'best_index_': 0,
  'mean_scores': [0.010716326271436491]},
 {'logn': 9,
  'model': 'st_0_under',
  'cv_results': None,
  'best_index_': 0,
  'mean_scores': [0.010128914898503861]},
 {'logn': 9,
  'model': 'st_0_not-under',
  'cv_results': None,
  'best_index_': 0,
  'mean_scores': [0.010128914898503861]},
 {'logn': 10,
  'model': 'st_0_under',
  'cv_results': None,
  'best_index_': 0,
  'mean_scores': [0.010007403941710203]},
 {'logn': 10,
  'model': 'st_0_not-under',
  'cv_results': None,
  'best_index_': 0,
  'mean_scores': [0.010007403941710203]},
 {'logn': 11,
  'model': 'st_0_under',
  'cv_results': None,
  'best_index_': 0,
  'mean_scores': [0.009886594880895829]},
 {'logn': 11,
  'model': 'st_0_not-under',
  'cv_results': None,
  'best_index_': 0,
  'mean_scores': [0.009886594880895829]},
 {'l

In [42]:
st_results = [result for result in results if 'st' in result['model']]
kt_results = [result for result in results if 'kt' in result['model']]

## Figure 1

In [43]:
row_subplot_titles = [
    # "Test Score vs n", 
    # "Log10 Test Score vs n", 
    'Log10 Excess Risk vs m'
]

fig = make_subplots(
    rows=len(row_subplot_titles),
    cols=len(varying_variable_values),
    shared_yaxes=True,
    subplot_titles=reduce(concat, [[f'{varying_variable}={v}' for v in varying_variable_values] for _ in row_subplot_titles]),
    vertical_spacing=0.1,
)

# model_names = list(model_configs.keys())
colors_list = colors.qualitative.Plotly * 4
colors_used = set()

In [44]:
def plot_vs_n(fig, results, print_name, vvv, r, c, is_better='higher', scale='log2', dash='solid'):
    """
    Plot Figure 1 from DnC paper
    - x-axis: log2(n)
    - y-axis: test score (or excess risk)
    - different lines correspond to different values of m

    Args:
    - vvv: varying variable value
    """
    result_m = {m: [] for m in range(0, logn_hi, 2)}
    
    for result in results:
        model_name, m, vv_name = result["model"].split('_')

        # only select results with the correct varying variable value
        if vv_name != vvv:
            continue

        # color = colors_list[model_names.index(f'{model_name}_{m}')]
        color = colors_list[int(m)//2]

        if scale == 'log2':
            y = np.log2(np.abs(result[f"mean_scores"]))
            hline = np.log2(np.abs(baseline_loss))

        elif scale == 'log10':
            y = np.log10(np.abs(result[f"mean_scores"]))
            hline = np.log10(np.abs(baseline_loss))

        elif scale == 'linear':
            hline = np.abs(baseline_loss)
            y = np.abs(result[f"mean_scores"])

        elif scale == 'excess-log10':
            y = np.log10(np.abs(result[f"mean_scores"]) - np.abs(baseline_loss))
            hline = 0

        trace = go.Box(
            x=[result['logn']]*len(result[f"mean_scores"]),
            y=y,
            name=f'm={m}',
            # opacity=0.5,
            legendgroup=str(m),
            line_color=color,
            offsetgroup=str(m),
            showlegend=color not in colors_used,
            boxmean=True,
            # line=dict(
            #     width=1,
            #     color=color,
            #     dash=dash,
            # ),
        )

        fig.add_trace(trace, row=r, col=c)
        colors_used.add(color)

        result_m[int(m)].append((result['logn'], np.mean(y), np.std(y)))

    if 'excess' not in scale:
        # add line for baseline loss
        fig.add_hline(
            y=hline,
            row=r, col=c, line_dash="dash",
        )

    if c == 1: fig.update_yaxes(title_text=f"{scale}({print_name}) - {is_better} is better", row=r, col=c)
    fig.update_xaxes(title_text="log2(n)", type='linear', row=r, col=c)
    fig.update_yaxes(type='linear', row=r, col=c)
    fig.update_layout(boxmode='group')

    return result_m

In [45]:
st_result_m = {}
kt_result_m = {}
for c, vvv in enumerate(varying_variable_values):
    # plot_test_score_vs_n(str(vvv), 1, c+1, scale='linear')
    # plot_test_score_vs_n(str(vvv), 2, c+1, scale='log10')
    # result_m[vvv] = plot_test_score_vs_n(str(vvv), 3, c+1, scale='excess-log10')
    st_result_m[vvv] = plot_vs_n(fig, st_results, 'st', str(vvv), 1, c+1, scale='excess-log10', dash='solid')
    kt_result_m[vvv] = plot_vs_n(fig, kt_results, 'kt', str(vvv), 1, c+1, scale='excess-log10', dash='dot')

fig = make_subplots(
    rows=1,
    cols=len(varying_variable_values),
    shared_yaxes=True,
)

for c, vvv in enumerate(varying_variable_values):
    
    for m, result in st_result_m[vvv].items():
        x,y, y_std = zip(*st_result_m[vvv][m])
        fig.add_trace(go.Scatter(
            x=x, y=y, 
            mode='lines+markers', 
            name=f'm={m}', 
            error_y=dict(type='data', array=y_std, visible=True),
            legendgroup=str(m),
            # line_color = colors_list[m//2]
            line=dict(
                width=1,
                color=colors_list[m//2],
                dash='solid',
            ),
        ), row=1, col=c+1)

    for m, result in kt_result_m[vvv].items():
        x,y, y_std = zip(*kt_result_m[vvv][m])
        fig.add_trace(go.Scatter(
            x=x, y=y, 
            mode='lines+markers', 
            name=f'm={m}', 
            error_y=dict(type='data', array=y_std, visible=True),
            legendgroup=str(m),
            # line_color = colors_list[m//2],
            line=dict(
                width=1,
                color=colors_list[m//2],
                dash='dot',
            ),
        ), row=1, col=c+1)

# update x axis title
fig.update_xaxes(title_text="logn")
fig.update_yaxes(title_text="log10(Excess Risk)", row=1, col=1)
# fig.update_layout(title=f'Figure 1: f={f_name}(sigma={noise})')
fig.show()

/tmp/ipykernel_3130494/768042693.py:36: RuntimeWarning:

invalid value encountered in log10



## Figure 2

In [46]:
row_subplot_titles = [
    # "Test Score vs m", 
    # "Log10 Test Score vs m", 
    'Log10 Excess Risk vs m'
] #, "Train time vs n", "Predict time vs n"]

fig = make_subplots(
    rows=len(row_subplot_titles),
    cols=1, #len(varying_variable_values),
    shared_yaxes=True,
    # subplot_titles=reduce(concat, [[f'{varying_variable}={v}' for v in varying_variable_values] for _ in row_subplot_titles]),
    subplot_titles=row_subplot_titles,
    vertical_spacing=0.1,
)

# model_names = list(model_configs.keys())
colors_list = colors.qualitative.Plotly * (
    (logn_hi-logn_lo) // len(colors.qualitative.Plotly) + 1
)
colors_used = set()

In [47]:
def plot_vs_m(fig, results, vvv, r, c, is_better='higher', scale='log2'):
    """
    Plot Figure 2 from DnC paper
    - x-axis: m
    - y-axis: test score (or excess risk)
    - different lines correspond to different values of logn

    Args:
    - vvv: varying variable value
    """
    result_logn = {logn: [] for logn in range(logn_lo, logn_hi)}
    
    for result in results:
        model_name, m, vv_name = result["model"].split('_')

        # only select results with the correct varying variable value
        if vv_name != vvv:
            continue
        
        logn = result['logn']
        color = colors_list[logn - logn_lo]
        # print(model_name, m, f'logn: {logn}')   
        
        if scale == 'log2':
            y = np.log2(np.abs(result[f"mean_scores"]))
            hline = np.log2(np.abs(baseline_loss))

        elif scale == 'log10':
            y = np.log10(np.abs(result[f"mean_scores"]))
            hline = np.log10(np.abs(baseline_loss))

        elif scale == 'excess-log10':
            y = np.log10(np.abs(result[f"mean_scores"]) - np.abs(baseline_loss))
            hline = 0

        elif scale == 'linear':
            hline = np.abs(baseline_loss)
            y = np.abs(result[f"mean_scores"])

        trace = go.Box(
            x=[m,]*len(result[f"mean_scores"]),
            y=y,
            name=f'logn={logn}',
            # opacity=0.5,
            legendgroup=str(logn),
            line_color=color,
            offsetgroup=str(logn),
            showlegend=color not in colors_used,
            boxmean=True,
        )

        fig.add_trace(trace, row=r, col=c)
        colors_used.add(color)

        result_logn[logn].append((int(m)/logn, np.mean(y), np.std(y)))

    # add line for baseline loss
    if 'excess' not in scale:
        fig.add_hline(
            y=hline,
            row=r, col=c, line_dash="dash",
        )

    # if c == 1: fig.update_yaxes(title_text=f"{scale}({print_name}) - {is_better} is better", row=r, col=c)
    fig.update_xaxes(title_text="m", type='linear', row=r, col=c)
    fig.update_yaxes(type='linear', row=r, col=c)
    fig.update_layout(boxmode='group')

    return result_logn

In [48]:
for c, vvv in enumerate(['under',]):
    # plot_test_score_vs_m(str(vvv), 1, c+1, scale='linear')
    # plot_test_score_vs_m(str(vvv), 2, c+1, scale='log10')
    st_result_logn = plot_vs_m(fig, st_results, str(vvv), 1, c+1, scale='excess-log10')
    kt_result_logn = plot_vs_m(fig, kt_results, str(vvv), 1, c+1, scale='excess-log10')

fig = go.Figure()
for logn, result in st_result_logn.items():
    x,y, y_std = zip(*st_result_logn[logn])
    fig.add_trace(go.Scatter(
        x=x, 
        y=y, 
        mode='lines+markers', 
        name=f'N={2**logn}', 
        error_y=dict(
            type='data', 
            array=y_std, 
            visible=True
        ),
        line=dict(
            width=1,
            color=colors_list[logn - logn_lo],
            dash='solid',
        ),
        legendgroup=str(logn),
    ))
for logn, result in kt_result_logn.items():
    x,y, y_std = zip(*kt_result_logn[logn])
    fig.add_trace(go.Scatter(
        x=x, 
        y=y, 
        mode='lines+markers', 
        name=f'N={2**logn}', 
        error_y=dict(
            type='data', 
            array=y_std, 
            visible=True
        ),
        line=dict(
            width=1,
            color=colors_list[logn - logn_lo],
            dash='dot',
        ),
        legendgroup=str(logn),
    ))

# update x axis title
fig.update_xaxes(title_text="m/logn")
fig.update_yaxes(title_text="log10(Excess Risk)")
# fig.update_layout(title=f'Figure 2: f={f_name}(sigma={noise})')
fig.show()

/tmp/ipykernel_3130494/3446466205.py:33: RuntimeWarning:

invalid value encountered in log10

